In [1]:
%load_ext autoreload
%autoreload 2

# Add project root folder to module loading paths.
import sys
sys.path.append('..')

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta
import pytz
from zipfile import ZipFile
from pathlib import Path

from src.utils import remake_folder 

### Download dos dados históricos
O Download dos dados históricos podem ser realizados no endereço abaixo, infelizmente, de forma manual por enquanto, visto que exigem um captcha para serem baixados.

Salvar os arquivos no local definido em `LOCAL_PATH_B3_DOWNLOAD`

http://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/

In [3]:
INITIAL_YEAR = 2020
FINAL_YEAR = 2021

LOCAL_PATH_B3 = '../data/b3/cotacoes_historicas'

# Local onde os arquivos históricos da b3 foram salvos
LOCAL_PATH_B3_DOWNLOAD = '../data/b3/cotacoes_historicas/download'

# Local onde os aruivos extraidos baixados da B3 serão salvos, a extração será realizada pelo script
LOCAL_PATH_B3_ARQUIVOS = '../data/b3/cotacoes_historicas/arquivos'


In [4]:
remake_folder(LOCAL_PATH_B3_ARQUIVOS)
for YEAR in range(INITIAL_YEAR, FINAL_YEAR + 1):
    ZipFile(LOCAL_PATH_B3_DOWNLOAD + "/COTAHIST_A" + str(YEAR) + ".ZIP").extractall(LOCAL_PATH_B3_ARQUIVOS)
    

In [5]:
raw_df = pd.DataFrame()
for YEAR in range(INITIAL_YEAR, FINAL_YEAR + 1):
    cur_df = pd.read_csv(LOCAL_PATH_B3_ARQUIVOS + "/COTAHIST_A" + str(YEAR) + ".TXT", header=None)
    
    # removes firs line, information related to the historical data generation file
    cur_df.drop(cur_df.head(1).index,inplace=True) # drop first n rows
    
    # removes last line, information related to the historical data generation
    cur_df.drop(cur_df.tail(1).index,inplace=True) # drop last n rows
    
    raw_df =  pd.concat([raw_df, cur_df], ignore_index=True)
    

In [6]:
raw_df.head()

,0
0,012020010202AALR3 010ALLIAR ON ...
1,012020010202AAPL34 010APPLE DRN ...
2,012020010202ABCB4 010ABC BRASIL PN EJ ...
3,012020010202ABEV3 010AMBEV S/A ON EJ ...
4,012020010202ADHM3 010ADVANCED-DH ON ...


In [7]:
raw_df.tail()

,0
1740198,012021041662WIZS3T 030WIZ S.A. ON ED ...
1740199,012021041662WIZS3T 030WIZ S.A. ON ED ...
1740200,012021041662WIZS3T 030WIZ S.A. ON ED ...
1740201,012021041662XINA11T 030TREND CHINA CI ...
1740202,012021041662YDUQ3T 030YDUQS PART ON ...


### Processamento dos dados

Os dados estão compactados em cada coluna em apenas uma string, aqui os dados são sepados em diferentes colunas.

Para mais detalhes sobre a organização das informações, a B3 fornece o seguinte documento, descrevendo o layout das informações presentes no arquivo histórico.

http://www.b3.com.br/data/files/33/67/B9/50/D84057102C784E47AC094EA8/SeriesHistoricas_Layout.pdf

In [8]:
data = []
for index, row in raw_df.iterrows():
    asset = []
    # TIPREG
    #asset.append(row[0][0:2].strip())
    # DATA
    asset.append(row[0][2:10].strip())
    # CODBDI
    #asset.append(row[0][10:12].strip())
    # CODNEG
    asset.append(row[0][12:24].strip())
    # TPMERC
    #asset.append(row[0][24:27].strip())
    # NOMRES
    #asset.append(row[0][27:39].strip())
    # ESPECI
    #asset.append(row[0][30:49].strip())
    # PRAZOT
    #asset.append(row[0][49:52].strip())
    # MOEDA
    asset.append(row[0][52:56].strip())
    # PREABE
    asset.append(row[0][56:69].strip())
    # PREMAX
    asset.append(row[0][69:82].strip())
    # PREMIN
    asset.append(row[0][82:95].strip())
    # PREMED
    asset.append(row[0][95:108].strip())
    # PREULT
    asset.append(row[0][108:121].strip())
    # PREOFC
    asset.append(row[0][121:134].strip())
    # PREOFV
    asset.append(row[0][134:147].strip())
    # TOTNEG
    asset.append(row[0][147:152].strip())
    # QUATOT
    asset.append(row[0][152:170].strip())
    # VOLTOT
    asset.append(row[0][179:188].strip())
    # PREEXE
    #asset.append(row[0][188:201].strip())
    # INDOPC
    #asset.append(row[0][201:202].strip())
    # DATVEN
    #asset.append(row[0][202:210].strip())
    # FATCOT
    asset.append(row[0][210:217].strip())
    # PTOEXE
    #asset.append(row[0][217:230].strip())
    # CODISI
    #asset.append(row[0][230:242].strip())
    # DISMES
    #asset.append(row[0][242:245].strip())
    data.append(asset)


In [9]:
df = pd.DataFrame(data, columns = [
    #'TIPREG',
    'DATA',
    #'CODBDI',
    'CODNEG',
    #'TPMERC',
    #'NOMRES',
    #'ESPECI',
    #'PRAZOT',
    'MOEDA',
    'PREABE',
    'PREMAX',
    'PREMIN',
    'PREMED',
    'PREULT',
    'PREOFC',
    'PREOFV',
    'TOTNEG',
    'QUATOT',
    'VOLTOT',
    #'PREEXE',
    #'INDOPC',
    #'DATVEN',
    'FATCOT',
    #'PTOEXE',
    #'CODISI',
    #'DISMES',
]) 


Para facilitar a leitura das informações, as colunas foram renomeadas com os seguintes nomes

In [10]:
df.columns = [
    #'TIPREG',
    'date',
    #'cod_bdi',
    'cod_negociacao',
    #'tipo_mercado',
    #'nm_resumido',
    #'epecificacao_papel',
    #'PRAZOT',
    'moeda',
    'preco_abertura',
    'preco_maximo',
    'preco_minimo',
    'preco_medio',
    'preco_ultimo',
    'preco_melhor_oferta_compra',
    'preco_melhor_oferta_venda',
    'total_negocios',
    'total_titulos_negociados',
    'volume_titulos_negociados',
    #'PREEXE',
    #'INDOPC',
    #'DATVEN',
    'fator_cotacao',
    #'PTOEXE',
    #'isin',
    #'DISMES',
]

In [11]:
df.head()

,date,cod_negociacao,moeda,preco_abertura,preco_maximo,preco_minimo,preco_medio,preco_ultimo,preco_melhor_oferta_compra,preco_melhor_oferta_venda,total_negocios,total_titulos_negociados,volume_titulos_negociados,fator_cotacao
0,20200102,AALR3,R$,0000000001829,0000000001900,0000000001828,0000000001868,0000000001900,0000000001899,0000000001901,02443,000000000000585800,094619600,0000001
1,20200102,AAPL34,R$,0000000012000,0000000012134,0000000012000,0000000012073,0000000012134,0000000010550,0000000012134,00009,000000000000012700,153338000,0000001
2,20200102,ABCB4,R$,0000000002000,0000000002030,0000000001982,0000000002005,0000000002030,0000000002008,0000000002030,03979,000000000000870400,745787800,0000001
3,20200102,ABEV3,R$,0000000001886,0000000001925,0000000001878,0000000001912,0000000001920,0000000001919,0000000001920,31581,000000000016011300,623479100,0000001
4,20200102,ADHM3,R$,0000000000255,0000000000255,0000000000235,0000000000244,0000000000248,0000000000248,0000000000249,00307,000000000000193400,047192700,0000001


Conversão dos campos de data em datetime, levando em consideração o horário de brasília (GMT-3)


In [12]:
diferenca = timedelta(hours=-3)
fuso_horario = timezone(diferenca)
print(fuso_horario)
# datetime.now(tz=fuso_horario).isoformat()

UTC-03:00


In [13]:

df['created_at'] = datetime.now(tz=fuso_horario).isoformat()
df['updated_at'] = datetime.now(tz=fuso_horario).isoformat()
df['date'] = df.date.astype(str).apply(lambda x: datetime.strptime(x , '%Y%m%d').replace(tzinfo=fuso_horario).isoformat())


In [14]:
df.head()

,date,cod_negociacao,moeda,preco_abertura,preco_maximo,preco_minimo,preco_medio,preco_ultimo,preco_melhor_oferta_compra,preco_melhor_oferta_venda,total_negocios,total_titulos_negociados,volume_titulos_negociados,fator_cotacao,created_at,updated_at
0,2020-01-02T00:00:00-03:00,AALR3,R$,0000000001829,0000000001900,0000000001828,0000000001868,0000000001900,0000000001899,0000000001901,02443,000000000000585800,094619600,0000001,2021-04-17T15:17:16.793795-03:00,2021-04-17T15:17:16.805792-03:00
1,2020-01-02T00:00:00-03:00,AAPL34,R$,0000000012000,0000000012134,0000000012000,0000000012073,0000000012134,0000000010550,0000000012134,00009,000000000000012700,153338000,0000001,2021-04-17T15:17:16.793795-03:00,2021-04-17T15:17:16.805792-03:00
2,2020-01-02T00:00:00-03:00,ABCB4,R$,0000000002000,0000000002030,0000000001982,0000000002005,0000000002030,0000000002008,0000000002030,03979,000000000000870400,745787800,0000001,2021-04-17T15:17:16.793795-03:00,2021-04-17T15:17:16.805792-03:00
3,2020-01-02T00:00:00-03:00,ABEV3,R$,0000000001886,0000000001925,0000000001878,0000000001912,0000000001920,0000000001919,0000000001920,31581,000000000016011300,623479100,0000001,2021-04-17T15:17:16.793795-03:00,2021-04-17T15:17:16.805792-03:00
4,2020-01-02T00:00:00-03:00,ADHM3,R$,0000000000255,0000000000255,0000000000235,0000000000244,0000000000248,0000000000248,0000000000249,00307,000000000000193400,047192700,0000001,2021-04-17T15:17:16.793795-03:00,2021-04-17T15:17:16.805792-03:00


In [15]:
df.preco_abertura = pd.to_numeric(df.preco_abertura, errors='coerce')/100
df.preco_maximo = pd.to_numeric(df.preco_maximo, errors='coerce')/100
df.preco_minimo = pd.to_numeric(df.preco_minimo, errors='coerce')/100
df.preco_medio = pd.to_numeric(df.preco_medio, errors='coerce')/100
df.preco_ultimo = pd.to_numeric(df.preco_ultimo, errors='coerce')/100
df.preco_melhor_oferta_compra = pd.to_numeric(df.preco_melhor_oferta_compra, errors='coerce')/100
df.preco_melhor_oferta_venda = pd.to_numeric(df.preco_melhor_oferta_venda, errors='coerce')/100
df.total_negocios = pd.to_numeric(df.total_negocios, errors='coerce')
df.total_titulos_negociados = pd.to_numeric(df.total_titulos_negociados, errors='coerce')
df.volume_titulos_negociados = pd.to_numeric(df.volume_titulos_negociados, errors='coerce')/100
df.fator_cotacao = pd.to_numeric(df.fator_cotacao, errors='coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1740203 entries, 0 to 1740202
Data columns (total 16 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   date                        object 
 1   cod_negociacao              object 
 2   moeda                       object 
 3   preco_abertura              float64
 4   preco_maximo                float64
 5   preco_minimo                float64
 6   preco_medio                 float64
 7   preco_ultimo                float64
 8   preco_melhor_oferta_compra  float64
 9   preco_melhor_oferta_venda   float64
 10  total_negocios              int64  
 11  total_titulos_negociados    int64  
 12  volume_titulos_negociados   float64
 13  fator_cotacao               int64  
 14  created_at                  object 
 15  updated_at                  object 
dtypes: float64(8), int64(3), object(5)
memory usage: 212.4+ MB


In [16]:
df.head()

,date,cod_negociacao,moeda,preco_abertura,preco_maximo,preco_minimo,preco_medio,preco_ultimo,preco_melhor_oferta_compra,preco_melhor_oferta_venda,total_negocios,total_titulos_negociados,volume_titulos_negociados,fator_cotacao,created_at,updated_at
0,2020-01-02T00:00:00-03:00,AALR3,R$,18.29,19.00,18.28,18.68,19.00,18.99,19.01,2443,585800,946196.0,1,2021-04-17T15:17:16.793795-03:00,2021-04-17T15:17:16.805792-03:00
1,2020-01-02T00:00:00-03:00,AAPL34,R$,120.00,121.34,120.00,120.73,121.34,105.50,121.34,9,12700,1533380.0,1,2021-04-17T15:17:16.793795-03:00,2021-04-17T15:17:16.805792-03:00
2,2020-01-02T00:00:00-03:00,ABCB4,R$,20.00,20.30,19.82,20.05,20.30,20.08,20.30,3979,870400,7457878.0,1,2021-04-17T15:17:16.793795-03:00,2021-04-17T15:17:16.805792-03:00
3,2020-01-02T00:00:00-03:00,ABEV3,R$,18.86,19.25,18.78,19.12,19.20,19.19,19.20,31581,16011300,6234791.0,1,2021-04-17T15:17:16.793795-03:00,2021-04-17T15:17:16.805792-03:00
4,2020-01-02T00:00:00-03:00,ADHM3,R$,2.55,2.55,2.35,2.44,2.48,2.48,2.49,307,193400,471927.0,1,2021-04-17T15:17:16.793795-03:00,2021-04-17T15:17:16.805792-03:00


In [17]:
# Ex: 
df.loc[(df['cod_negociacao'] == "SQIA3")][["date","cod_negociacao", "preco_ultimo", "fator_cotacao"]]

,date,cod_negociacao,preco_ultimo,fator_cotacao
316,2020-01-02T00:00:00-03:00,SQIA3,24.60,1
3007,2020-01-03T00:00:00-03:00,SQIA3,24.48,1
3008,2020-01-06T00:00:00-03:00,SQIA3,25.00,1
7069,2020-01-07T00:00:00-03:00,SQIA3,25.10,1
7070,2020-01-08T00:00:00-03:00,SQIA3,25.09,1
...,...,...,...,...
1351566,2021-04-12T00:00:00-03:00,SQIA3,22.66,1
1353102,2021-04-13T00:00:00-03:00,SQIA3,22.39,1
1354606,2021-04-14T00:00:00-03:00,SQIA3,22.24,1
1356144,2021-04-15T00:00:00-03:00,SQIA3,22.22,1


In [18]:
#save to json

df.to_json(f'{LOCAL_PATH_B3}/cotacoes_{str(INITIAL_YEAR)}_{str(FINAL_YEAR)}.json', orient='records')

In [19]:
# TODO - Outros arquivos para tratar na sequência, aqui ou em outros scripts
# https://arquivos.b3.com.br/apinegocios/tickercsv/2020-06-05
# http://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/cotacoes/cotacoes/
# https://arquivos.b3.com.br/Web/Consolidated
